In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:
dataset=fetch_20newsgroups(remove=('headers','footers','quotes'))

In [4]:
doc=dataset.data
len(doc)

11314

In [5]:
y=dataset.target

In [6]:
len(y)

11314

In [7]:
new_y=[]
new_doc=[]
for i in range(len(y)):
    if y[i] in [11,12,13,14]:
        new_y.append(y[i]-10)
    else:
        continue
    new_doc.append(doc[i])
y=new_y
doc=new_doc

In [8]:
df=pd.DataFrame({'doc':doc})
df['clean']=df['doc'].str.replace("[^a-zA-Z#]", " ")
df['clean']=df['clean'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
df['clean']=df['clean'].apply(lambda x: x.lower())

In [9]:
from nltk.corpus import stopwords
df['clean'].head()

0    from article owcb world tombaker world baker u...
1    there were people responded request info treat...
2    description external tank option redesign dele...
3    would like able amplify voltage signal which o...
4    actually fossil fuel plants hotter than usual ...
Name: clean, dtype: object

In [10]:
sw=stopwords.words('english')

tok=df['clean'].apply(lambda x:x.split())
tok=tok.apply(lambda x: [item for item in x if item not in sw])

In [11]:
detok=[]

for i in range(len(df)):
    t=' '.join(tok[i])
    detok.append(t)
    

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train,x_test,y_train,y_test=train_test_split(df['clean'],y,test_size=0.2)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

In [15]:
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,
                             min_df=2, stop_words='english',
                             use_idf=True)


In [16]:
x_train_tfidf=vectorizer.fit_transform(x_train)
x_test_tfidf=vectorizer.transform(x_test)
svd=TruncatedSVD(n_components=50)
lsa=make_pipeline(svd,Normalizer(copy=False))

In [17]:
x_train_lsa=lsa.fit_transform(x_train_tfidf)


x_test_lsa=lsa.transform(x_test_tfidf)

In [1]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [19]:
clf=SVC(C=2)
clf.fit(x_train_lsa,y_train)

C:\Users\KRISHNA\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [20]:
pre=clf.predict(x_test_lsa)
print(classification_report(y_pred=pre,y_true=y_test))

              precision    recall  f1-score   support

           1       0.87      0.82      0.84       109
           2       0.72      0.90      0.80       112
           3       0.88      0.78      0.83       119
           4       0.83      0.78      0.80       135

    accuracy                           0.82       475
   macro avg       0.82      0.82      0.82       475
weighted avg       0.83      0.82      0.82       475

